# Credit Markets Final Exam – Consolidated, Step-by-Step Solutions
This notebook restates every exam question and solves it in order using the provided data and helper utilities.
Data manipulations are shown as data frames, Plotly provides interactive visuals, and latex-formatted tables summarize
key outputs.  All calculations are rerunnable from a clean environment.

In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import QuantLib as ql
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam import credit_market_tools as cmt

pd.set_option('display.float_format', lambda v: f"{v:,.4f}")
BASE = Path('UChicago_FINM_35700_CreditMarkets_Spring2024_FinalExam')
DATA = BASE / 'data'
calc_date = pd.Timestamp('2024-05-03')
ql_calc_date = ql.Date(calc_date.day, calc_date.month, calc_date.year)
ql.Settings.instance().evaluationDate = ql_calc_date

def latex_table(df):
    align = '|'.join(['c'] * len(df.columns))
    lines = [r"\begin{document}", rf"\begin{{tabular}}{{|{align}|}}", r"\hline"]
    lines.append(' & '.join(df.columns) + r" \\ \hline")
    for _, row in df.iterrows():
        lines.append(' & '.join([str(v) for v in row]) + r" \\ \hline")
    lines.extend([r"\end{tabular}", r"\end{document}"])
    return "```latex\n" + "\n".join(lines) + "\n```"

## Problem 1 – Overall understanding of credit models (True/False)
Ceteris paribus assumptions apply. Each table below directly mirrors the four sub-questions.

In [2]:
problem1_answers = {
    "1a_prices": [
        ("Price vs interest rate", True),
        ("Price vs hazard rate", True),
        ("Price vs expected recovery", False),
        ("Price vs coupon", False),
        ("Price vs maturity", True),
    ],
    "1b_yields": [
        ("Yield vs interest rate", False),
        ("Yield vs hazard rate", False),
        ("Yield vs expected recovery", True),
        ("Yield vs coupon", False),
        ("Yield vs maturity", False),
    ],
    "1c_merton": [
        ("Equity value vs assets", False),
        ("Equity vol vs assets", True),
        ("Equity value vs asset vol", False),
        ("Equity value vs liabilities", True),
        ("Equity vol vs liabilities", False),
    ],
    "1d_spreads": [
        ("Yield vs liabilities", False),
        ("Expected recovery vs liabilities", True),
        ("Yield vs asset vol", False),
        ("Credit spread vs assets", True),
        ("Credit spread vs asset vol", False),
    ],
}

problem1_tables = {k: pd.DataFrame(v, columns=["Statement", "True?"]) for k, v in problem1_answers.items()}
for key, df in problem1_tables.items():
    display(df)
    print(latex_table(df))

,Statement,True?
0,Price vs interest rate,True
1,Price vs hazard rate,True
2,Price vs expected recovery,False
3,Price vs coupon,False
4,Price vs maturity,True


```latex
\begin{document}
\begin{tabular}{|c|c|}
\hline
Statement & True? \\ \hline
Price vs interest rate & True \\ \hline
Price vs hazard rate & True \\ \hline
Price vs expected recovery & False \\ \hline
Price vs coupon & False \\ \hline
Price vs maturity & True \\ \hline
\end{tabular}
\end{document}
```


,Statement,True?
0,Yield vs interest rate,False
1,Yield vs hazard rate,False
2,Yield vs expected recovery,True
3,Yield vs coupon,False
4,Yield vs maturity,False


```latex
\begin{document}
\begin{tabular}{|c|c|}
\hline
Statement & True? \\ \hline
Yield vs interest rate & False \\ \hline
Yield vs hazard rate & False \\ \hline
Yield vs expected recovery & True \\ \hline
Yield vs coupon & False \\ \hline
Yield vs maturity & False \\ \hline
\end{tabular}
\end{document}
```


,Statement,True?
0,Equity value vs assets,False
1,Equity vol vs assets,True
2,Equity value vs asset vol,False
3,Equity value vs liabilities,True
4,Equity vol vs liabilities,False


```latex
\begin{document}
\begin{tabular}{|c|c|}
\hline
Statement & True? \\ \hline
Equity value vs assets & False \\ \hline
Equity vol vs assets & True \\ \hline
Equity value vs asset vol & False \\ \hline
Equity value vs liabilities & True \\ \hline
Equity vol vs liabilities & False \\ \hline
\end{tabular}
\end{document}
```


,Statement,True?
0,Yield vs liabilities,False
1,Expected recovery vs liabilities,True
2,Yield vs asset vol,False
3,Credit spread vs assets,True
4,Credit spread vs asset vol,False


```latex
\begin{document}
\begin{tabular}{|c|c|}
\hline
Statement & True? \\ \hline
Yield vs liabilities & False \\ \hline
Expected recovery vs liabilities & True \\ \hline
Yield vs asset vol & False \\ \hline
Credit spread vs assets & True \\ \hline
Credit spread vs asset vol & False \\ \hline
\end{tabular}
\end{document}
```


## Problem 2 – AAPL fixed-rate corporate bond (US037833AT77)
*2a* build the bond object and list cash flows.  *2b* compute analytic price/DV01/duration/convexity from mid-yield.
*2c–d* run scenario curves from 2%–10% in 0.5% steps.

In [3]:
bond_sym = pd.read_excel(DATA / 'bond_symbology.xlsx')
bond_mkt = pd.read_excel(DATA / 'bond_market_prices_eod.xlsx')

AAPL_ISIN = 'US037833AT77'
aapl_sym = bond_sym[bond_sym['isin'] == AAPL_ISIN].iloc[0]
aapl_mkt = bond_mkt[bond_mkt['isin'] == AAPL_ISIN].iloc[0]
aapl_mid_yield = float(np.mean([aapl_mkt['bidYield'], aapl_mkt['askYield']]))

fixed_rate_bond = cmt.create_bond_from_symbology(aapl_sym)
cashflows_df = cmt.get_bond_cashflows(fixed_rate_bond, ql_calc_date)
cashflows_df

,CashFlowDate,CashFlowYearFrac,CashFlowAmount
19,"May 6th, 2024",0.0083,2.2250
20,"November 6th, 2024",0.5083,2.2250
21,"May 6th, 2025",1.0083,2.2250
22,"November 6th, 2025",1.5083,2.2250
23,"May 6th, 2026",2.0083,2.2250
24,"November 6th, 2026",2.5083,2.2250
25,"May 6th, 2027",3.0083,2.2250
26,"November 6th, 2027",3.5083,2.2250
27,"May 6th, 2028",4.0083,2.2250
28,"November 6th, 2028",4.5083,2.2250


In [4]:
bond_dc = fixed_rate_bond.dayCounter()
yield_decimal = aapl_mid_yield / 100
rate = ql.InterestRate(yield_decimal, bond_dc, ql.Compounded, ql.Semiannual)

price = ql.BondFunctions.cleanPrice(fixed_rate_bond, yield_decimal, bond_dc, ql.Compounded, ql.Semiannual)
duration = ql.BondFunctions.duration(fixed_rate_bond, rate)
convexity = ql.BondFunctions.convexity(fixed_rate_bond, yield_decimal, bond_dc, ql.Compounded, ql.Semiannual)
bond_dv01 = ql.BondFunctions.bps(fixed_rate_bond, yield_decimal, bond_dc, ql.Compounded, ql.Semiannual)

aapl_metrics = pd.DataFrame({
    'mid_yield': [aapl_mid_yield],
    'price': [price],
    'duration': [duration],
    'convexity': [convexity],
    'dv01': [bond_dv01]
})
display(aapl_metrics)
print(latex_table(aapl_metrics))

,mid_yield,price,duration,convexity,dv01
0,5.1175,91.7047,12.7916,217.6269,0.1243


```latex
\begin{document}
\begin{tabular}{|c|c|c|c|c|}
\hline
mid_yield & price & duration & convexity & dv01 \\ \hline
5.1175 & 91.70474876510454 & 12.791579838704548 & 217.62690365865657 & 0.12429854028477637 \\ \hline
\end{tabular}
\end{document}
```


In [5]:
scenario_yields = np.arange(0.02, 0.1001, 0.005)
scenario_prices = []
scenario_durations = []
scenario_convexities = []

for y in scenario_yields:
    price_y = ql.BondFunctions.cleanPrice(fixed_rate_bond, y, bond_dc, ql.Compounded, ql.Semiannual)
    dur_y = ql.BondFunctions.duration(fixed_rate_bond, ql.InterestRate(y, bond_dc, ql.Compounded, ql.Semiannual))
    conv_y = ql.BondFunctions.convexity(fixed_rate_bond, y, bond_dc, ql.Compounded, ql.Semiannual)
    scenario_prices.append(price_y)
    scenario_durations.append(dur_y)
    scenario_convexities.append(conv_y)

scenario_df = pd.DataFrame({
    'yield_pct': scenario_yields * 100,
    'price': scenario_prices,
    'duration': scenario_durations,
    'convexity': scenario_convexities,
})

fig_price = px.line(scenario_df, x='yield_pct', y='price', title='AAPL Price vs Yield', markers=True)
fig_duration = px.line(scenario_df, x='yield_pct', y='duration', title='AAPL Duration vs Yield', markers=True)
fig_convexity = px.line(scenario_df, x='yield_pct', y='convexity', title='AAPL Convexity vs Yield', markers=True)

scenario_df.head(), fig_price, fig_duration, fig_convexity

(   yield_pct    price  duration  convexity
 0     2.0000 140.2179   14.4286   258.7252
 1     2.5000 130.5404   14.1740   252.1831
 2     3.0000 121.6867   13.9159   245.6091
 3     3.5000 113.5808   13.6546   239.0121
 4     4.0000 106.1543   13.3904   232.4014,
 Figure({
     'data': [{'hovertemplate': 'yield_pct=%{x}<br>price=%{y}<extra></extra>',
               'legendgroup': '',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines+markers',
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'type': 'scatter',
               'x': {'bdata': ('AAAAAAAAAEAAAAAAAAAEQAEAAAAAAA' ... 'AAACJAAQAAAAAAI0ABAAAAAAAkQA=='),
                     'dtype': 'f8'},
               'xaxis': 'x',
               'y': {'bdata': ('Kain4fiGYUCSgoQGS1FgQAnlMsXya1' ... 'OyEU1A/jstKDuTS0Czf3T4UzFKQA=='),
                     'dtype': 'f8'},
               'yaxis'

## Problem 3 – Ford CDS curve calibration and valuation
*3a* bootstrap SOFR zero/discount curves. *3b* plot historical Ford CDS par spreads. *3c* calibrate Ford hazard curve as of
2024-05-03. *3d* value a 100 bps CDS maturing 2029-06-20.

In [6]:
sofr_sym = pd.read_excel(DATA / 'sofr_swaps_symbology.xlsx')
sofr_mkt = pd.read_excel(DATA / 'sofr_swaps_market_data_eod.xlsx')
sofr_details = sofr_sym.merge(sofr_mkt, on='figi').sort_values('tenor').drop_duplicates(subset='tenor')
sofr_curve = cmt.calibrate_sofr_curve_from_frame(ql_calc_date, sofr_details, 'midRate')
sofr_df = cmt.get_yield_curve_details_df(sofr_curve)

fig_sofr_zero = px.line(sofr_df, x='YearFrac', y='ZeroRate', title='SOFR Zero Rates')
fig_sofr_df = px.line(sofr_df, x='YearFrac', y='DiscountFactor', title='SOFR Discount Factors')
sofr_df.head(), fig_sofr_zero, fig_sofr_df

(         Date  YearFrac  DiscountFactor  ZeroRate
 0  2024-05-07    0.0000          1.0000    4.7990
 1  2025-05-07    1.0140          0.9540    4.7990
 2  2026-05-07    2.0280          0.9210    4.1260
 3  2027-05-07    3.0420          0.8930    3.8080
 4  2029-05-07    5.0720          0.8370    3.5660,
 Figure({
     'data': [{'hovertemplate': 'YearFrac=%{x}<br>ZeroRate=%{y}<extra></extra>',
               'legendgroup': '',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'type': 'scatter',
               'x': {'bdata': ('AAAAAAAAAADTTWIQWDnwP9NNYhBYOQ' ... 'aBlUNLJEB56SYxCEw0QFYOLbKdbz5A'),
                     'dtype': 'f8'},
               'xaxis': 'x',
               'y': {'bdata': ('GQRWDi0yE0AZBFYOLTITQBsv3SQGgR' ... 'cW2c73C0B/arx0kxgMQOkmMQisHApA'),
                     'd

In [7]:
cds_df = pd.read_excel(DATA / 'cds_market_data_eod.xlsx')
ford_cds = cds_df[cds_df['ticker'] == 'F']
spread_cols = ['par_spread_1y','par_spread_2y','par_spread_3y','par_spread_5y','par_spread_7y','par_spread_10y']

ford_long = ford_cds.melt(id_vars=['date'], value_vars=spread_cols, var_name='tenor', value_name='par_spread_bps')
fig_cds_hist = px.line(ford_long, x='date', y='par_spread_bps', color='tenor', title='Ford CDS Par Spreads History')
ford_long.head(), fig_cds_hist

(        date          tenor  par_spread_bps
 0 2024-01-02  par_spread_1y         65.6078
 1 2024-01-03  par_spread_1y         67.4076
 2 2024-01-04  par_spread_1y         65.9798
 3 2024-01-05  par_spread_1y         66.2923
 4 2024-01-08  par_spread_1y         64.9922,
 Figure({
     'data': [{'hovertemplate': 'tenor=par_spread_1y<br>date=%{x}<br>par_spread_bps=%{y}<extra></extra>',
               'legendgroup': 'par_spread_1y',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
               'name': 'par_spread_1y',
               'orientation': 'v',
               'showlegend': True,
               'type': 'scatter',
               'x': array(['2024-01-02T00:00:00.000000000', '2024-01-03T00:00:00.000000000',
                           '2024-01-04T00:00:00.000000000', '2024-01-05T00:00:00.000000000',
                           '2024-01-08T00:00:00.000000000', '2024-01-09T00:00:00.000000000',
  

In [8]:
ford_spread_today = ford_cds[ford_cds['date'] == calc_date].iloc[0]
par_spreads_vector = [ford_spread_today[c] for c in spread_cols]
sofr_handle = ql.YieldTermStructureHandle(sofr_curve)
hazard_curve = cmt.calibrate_cds_hazard_rate_curve(ql_calc_date, sofr_handle, par_spreads_vector, cds_recovery_rate=0.4)
hazard_df = cmt.get_hazard_rates_df(hazard_curve)

fig_hazard = px.line(hazard_df, x='YearFrac', y='HazardRateBps', title='Ford Hazard Rates')
fig_surv = px.line(hazard_df, x='YearFrac', y='SurvivalProb', title='Ford Survival Probability')
hazard_df.head(), fig_hazard, fig_surv

(         Date  YearFrac  HazardRateBps  SurvivalProb
 0  2024-05-03    0.0000        45.2954        1.0000
 1  2025-06-20    1.1472        45.2954        0.9949
 2  2026-06-22    2.1667       150.4952        0.9799
 3  2027-06-21    3.1778       262.9633        0.9546
 4  2029-06-20    5.2056       451.9982        0.8721,
 Figure({
     'data': [{'hovertemplate': 'YearFrac=%{x}<br>HazardRateBps=%{y}<extra></extra>',
               'legendgroup': '',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'type': 'scatter',
               'x': {'bdata': 'AAAAAAAAAACwBVuwBVvyP1VVVVVVVQFAwRZswRZsCUAofdInfdIUQO/u7u7u7hxA5DiO4ziOJEA=',
                     'dtype': 'f8'},
               'xaxis': 'x',
               'y': {'bdata': 'MPe89tClRkAw97z20KVGQPtwpwDZz2JAyLZ2eWlvcECs72qG+D98QO0L5kTLNYFAX

In [9]:
maturity_date = ql.Date(20, 6, 2029)
schedule = ql.Schedule(ql_calc_date, maturity_date, ql.Period(ql.Quarterly), ql.TARGET(), ql.Following, ql.Following, ql.DateGeneration.TwentiethIMM, False)
cds = ql.CreditDefaultSwap(ql.Protection.Seller, 10_000_000, 0.01, schedule, ql.Following, ql.Actual360(), True, True)
spot_rate = sofr_curve.zeroRate(sofr_curve.referenceDate(), ql.Actual365Fixed(), ql.Compounded).rate()
cds_discount_curve = ql.FlatForward(ql_calc_date, ql.QuoteHandle(ql.SimpleQuote(spot_rate)), ql.Actual365Fixed())
cds_discount_handle = ql.YieldTermStructureHandle(cds_discount_curve)
engine = ql.IsdaCdsEngine(ql.DefaultProbabilityTermStructureHandle(hazard_curve), 0.4, cds_discount_handle)
cds.setPricingEngine(engine)

cds_metrics = {
    'cds_pv': cds.NPV(),
    'premium_leg_pv': cds.couponLegNPV(),
    'default_leg_pv': cds.defaultLegNPV(),
    'par_spread_bps': cds.fairSpread() * 1e4,
    'survival_to_maturity': hazard_curve.survivalProbability(maturity_date)
}
cds_df_metrics = pd.DataFrame([cds_metrics])
display(cds_df_metrics)
print(latex_table(cds_df_metrics))

,cds_pv,premium_leg_pv,default_leg_pv,par_spread_bps,survival_to_maturity
0,"-213,271.3602","440,030.3882","-653,301.7484",148.4674,0.8721


```latex
\begin{document}
\begin{tabular}{|c|c|c|c|c|}
\hline
cds_pv & premium_leg_pv & default_leg_pv & par_spread_bps & survival_to_maturity \\ \hline
-213271.36020094674 & 440030.3881882865 & -653301.7483892333 & 148.46741632527645 & 0.8720738986891792 \\ \hline
\end{tabular}
\end{document}
```


## Problem 4 – Sympy derivations for flat-yield risky bonds
We re-derive analytic PV and DV01 for zero-coupon and interest-only bonds, then solve for the coupon \(c^*\) that equalizes the PVs.

In [10]:
c_sym, y_sym, T_sym = sp.symbols('c y T', positive=True)

zero_coupon_pv = sp.exp(-y_sym * T_sym)
zero_coupon_dv01 = -sp.diff(zero_coupon_pv, y_sym) * 1e-4

interest_only_pv = (c_sym/2) * (1 - sp.exp(-y_sym * T_sym)) / (sp.exp(y_sym/2) - 1)
interest_only_dv01 = -sp.diff(interest_only_pv, y_sym) * 1e-4

zero_coupon_pv, zero_coupon_dv01, interest_only_pv, interest_only_dv01

(exp(-T*y),
 0.0001*T*exp(-T*y),
 c*(1 - exp(-T*y))/(2*(exp(y/2) - 1)),
 -5.0e-5*T*c*exp(-T*y)/(exp(y/2) - 1) + 2.5e-5*c*(1 - exp(-T*y))*exp(y/2)/(exp(y/2) - 1)**2)

In [11]:
zc_pv_func = sp.lambdify((c_sym, y_sym, T_sym), zero_coupon_pv, 'numpy')
zc_dv01_func = sp.lambdify((c_sym, y_sym, T_sym), zero_coupon_dv01, 'numpy')
io_pv_func = sp.lambdify((c_sym, y_sym, T_sym), interest_only_pv, 'numpy')
io_dv01_func = sp.lambdify((c_sym, y_sym, T_sym), interest_only_dv01, 'numpy')

c_val = 0.05
y_vals = np.linspace(0.01, 0.10, 30)
T_vals = np.arange(1, 21)
Y, T = np.meshgrid(y_vals, T_vals)

zc_pv_vals = zc_pv_func(c_val, Y, T)
zc_dv_vals = zc_dv01_func(c_val, Y, T)
io_pv_vals = io_pv_func(c_val, Y, T)
io_dv_vals = io_dv01_func(c_val, Y, T)

fig_zc_pv = go.Figure(data=[go.Surface(x=Y, y=T, z=zc_pv_vals)])
fig_zc_pv.update_layout(title='Zero Coupon PV Surface', scene=dict(xaxis_title='Yield', yaxis_title='Maturity', zaxis_title='PV'))

fig_zc_dv = go.Figure(data=[go.Surface(x=Y, y=T, z=zc_dv_vals)])
fig_zc_dv.update_layout(title='Zero Coupon DV01 Surface', scene=dict(xaxis_title='Yield', yaxis_title='Maturity', zaxis_title='DV01'))

fig_io_pv = go.Figure(data=[go.Surface(x=Y, y=T, z=io_pv_vals)])
fig_io_pv.update_layout(title='Interest Only PV Surface', scene=dict(xaxis_title='Yield', yaxis_title='Maturity', zaxis_title='PV'))

fig_io_dv = go.Figure(data=[go.Surface(x=Y, y=T, z=io_dv_vals)])
fig_io_dv.update_layout(title='Interest Only DV01 Surface', scene=dict(xaxis_title='Yield', yaxis_title='Maturity', zaxis_title='DV01'))

fig_zc_pv, fig_zc_dv, fig_io_pv, fig_io_dv

(Figure({
     'data': [{'type': 'surface',
               'x': {'bdata': ('exSuR+F6hD9+LHoO+9WKP0Aio2qKmJ' ... 'PmJ9MCuD+ZFsBgNs64P5qZmZmZmbk/'),
                     'dtype': 'f8',
                     'shape': '20, 30'},
               'y': {'bdata': ('AQEBAQEBAQEBAQEBAQEBAQEBAQEBAQ' ... 'QUFBQUFBQUFBQUFBQUFBQUFBQUFBQU'),
                     'dtype': 'i1',
                     'shape': '20, 30'},
               'z': {'bdata': ('/Zwr/Xyu7z8Ul5RVW5XvP0qvfp1NfO' ... 'QlL86cwz/nsOnFpG7CP8yBv6OqUsE/'),
                     'dtype': 'f8',
                     'shape': '20, 30'}}],
     'layout': {'scene': {'xaxis': {'title': {'text': 'Yield'}},
                          'yaxis': {'title': {'text': 'Maturity'}},
                          'zaxis': {'title': {'text': 'PV'}}},
                'template': '...',
                'title': {'text': 'Zero Coupon PV Surface'}}
 }),
 Figure({
     'data': [{'type': 'surface',
               'x': {'bdata': ('exSuR+F6hD9+LHoO+9WKP0Aio2qKmJ' ... 'PmJ9

In [12]:
coupon_star = sp.solve(sp.Eq(interest_only_pv, zero_coupon_pv), c_sym)[0]
coupon_star

2*(1 - exp(y/2))/(1 - exp(T*y))

## Problem 5 – LQD ETF basket DV01 analysis
*5a* load basket and symbology; summarize counts/face and yield stats. *5b* compute each bond DV01 and contribution.
*5c–d* bucket by underlying benchmark Treasury, aggregate metrics, and visualize exposures.

In [13]:
lqd_basket = pd.read_excel(DATA / 'lqd_basket_composition.xlsx')
lqd_sym = pd.read_excel(DATA / 'lqd_corp_symbology.xlsx')

lqd_combined = lqd_basket.merge(lqd_sym, on='isin', suffixes=('', '_sym'))
lqd_combined = lqd_combined.dropna(subset=['midYield'])

summary_stats = {
    'bond_count': [len(lqd_combined)],
    'face_notional_mean': [lqd_combined['face_notional'].mean()],
    'face_notional_median': [lqd_combined['face_notional'].median()],
    'yield_mean': [lqd_combined['midYield'].mean()],
    'yield_std': [lqd_combined['midYield'].std()],
}
summary_df = pd.DataFrame(summary_stats)
display(summary_df)
print(latex_table(summary_df))

,bond_count,face_notional_mean,face_notional_median,yield_mean,yield_std
0,2461,"10,646,601.7879","9,177,000.0000",5.5081,0.4266


```latex
\begin{document}
\begin{tabular}{|c|c|c|c|c|}
\hline
bond_count & face_notional_mean & face_notional_median & yield_mean & yield_std \\ \hline
2461.0 & 10646601.787891101 & 9177000.0 & 5.508121292157659 & 0.426554468087778 \\ \hline
\end{tabular}
\end{document}
```


In [14]:
bond_dv01_list = []
for idx, row in lqd_combined.iterrows():
    row_local = row.to_dict()
    row_local['class'] = str(row_local['class']).capitalize()
    bond_obj = cmt.create_bond_from_symbology(row_local)
    y = row['midYield'] / 100
    dc = bond_obj.dayCounter()
    dv01 = ql.BondFunctions.bps(bond_obj, y, dc, ql.Compounded, ql.Semiannual)
    bond_dv01_list.append(dv01)

lqd_combined = lqd_combined.assign(bond_DV01=bond_dv01_list)
lqd_combined['basket_DV01'] = lqd_combined['bond_DV01'] * lqd_combined['face_notional'] / 10000
lqd_combined.head()

,date,holdings_date,etf_ticker,isin,security,issuer,coupon,maturity,cpn_type,class,...,acc_first,maturity_sym,mty_typ,rank,amt_out,country,currency_sym,status,bond_DV01,basket_DV01
0,2024-05-03,2024-05-02,LQD,US03522AAJ97,ABIBB 4.9 02/01/46,ANHEUSER-BUSCH COMPANIES LLC 4.9 02/01/2046 (S...,4.9000,2046-02-01,FIXED,CORP,...,2019-02-01,2046-02-01,CALLABLE,Sr Unsecured,"9,518.9640",US,USD,ACTV,0.1281,"1,011.5497"
1,2024-05-03,2024-05-02,LQD,US126650CZ11,CVS 5.05 03/25/48,CVS HEALTH CORP 5.05 03/25/2048 (SENIOR),5.0500,2048-03-25,FIXED,CORP,...,2018-03-09,2048-03-25,CALLABLE,Sr Unsecured,"8,000.0000",US,USD,ACTV,0.1254,793.2888
2,2024-05-03,2024-05-02,LQD,US38141GFD16,GS 6 3/4 10/01/37,GOLDMAN SACHS GROUP INC/THE T2 6.75 10/01/2037...,6.7500,2037-10-01,FIXED,CORP,...,2007-10-03,2037-10-01,AT MATURITY,Subordinated,"5,476.0690",US,USD,ACTV,0.0926,441.1388
3,2024-05-03,2024-05-02,LQD,US87264ABF12,TMUS 3 7/8 04/15/30,T-MOBILE USA INC 3.875 04/15/2030 (SENIOR),3.8750,2030-04-15,FIXED,CORP,...,2021-04-15,2030-04-15,CALLABLE,Sr Unsecured,"7,000.0000",US,USD,ACTV,0.0510,266.1993
4,2024-05-03,2024-05-02,LQD,US716973AG71,PFE 5.3 05/19/53,PFIZER INVESTMENT ENTERPRISES PTE 5.3 05/19/20...,5.3000,2053-05-19,FIXED,CORP,...,2023-05-19,2053-05-19,CALLABLE,Sr Unsecured,"6,000.0000",SI,USD,ACTV,0.1462,694.6130


In [15]:
agg_cols = {
    'isin': 'count',
    'face_notional': 'sum',
    'basket_DV01': 'sum'
}
agg_df = lqd_combined.groupby('und_bench_tsy_isin').agg(agg_cols).rename(columns={'isin': 'basket_count'}).reset_index()
display(agg_df)
print(latex_table(agg_df))

,und_bench_tsy_isin,basket_count,face_notional,basket_DV01
0,US912810TV08,586,6816956000,"96,576.2865"
1,US912810TZ12,397,4395740000,"52,589.4010"
2,US91282CJZ59,645,6572948000,"49,579.5793"
3,US91282CKJ98,140,1099876000,"3,629.6995"
4,US91282CKK61,1,11124000,35.4789
5,US91282CKN01,21,191166000,"1,117.0985"
6,US91282CKP58,671,7113477000,"32,724.0702"


```latex
\begin{document}
\begin{tabular}{|c|c|c|c|}
\hline
und_bench_tsy_isin & basket_count & face_notional & basket_DV01 \\ \hline
US912810TV08 & 586 & 6816956000 & 96576.2864874929 \\ \hline
US912810TZ12 & 397 & 4395740000 & 52589.401046398074 \\ \hline
US91282CJZ59 & 645 & 6572948000 & 49579.579342866746 \\ \hline
US91282CKJ98 & 140 & 1099876000 & 3629.699545107017 \\ \hline
US91282CKK61 & 1 & 11124000 & 35.478936516988945 \\ \hline
US91282CKN01 & 21 & 191166000 & 1117.0985380046168 \\ \hline
US91282CKP58 & 671 & 7113477000 & 32724.070157758026 \\ \hline
\end{tabular}
\end{document}
```


In [16]:
bond_sym = pd.read_excel(DATA / 'bond_symbology.xlsx')
bench_sym = bond_sym[bond_sym['isin'].isin(agg_df['und_bench_tsy_isin'])][['isin','security','maturity']]
bench_sym = bench_sym.assign(TTM=(bench_sym['maturity'] - calc_date).dt.days / 365)
combined_bench = agg_df.merge(bench_sym, left_on='und_bench_tsy_isin', right_on='isin', how='left')
combined_bench = combined_bench.sort_values('TTM')

bar_count = px.bar(combined_bench, x='security', y='basket_count', title='Basket Count by Benchmark TSY')
bar_face = px.bar(combined_bench, x='security', y='face_notional', title='Face Notional by Benchmark TSY')
bar_dv01 = px.bar(combined_bench, x='security', y='basket_DV01', title='DV01 by Benchmark TSY')

combined_bench, bar_count, bar_face, bar_dv01

(  und_bench_tsy_isin  basket_count  face_notional  basket_DV01          isin  \
 4       US91282CKK61             1       11124000      35.4789  US91282CKK61   
 3       US91282CKJ98           140     1099876000   3,629.6995  US91282CKJ98   
 6       US91282CKP58           671     7113477000  32,724.0702  US91282CKP58   
 5       US91282CKN01            21      191166000   1,117.0985  US91282CKN01   
 2       US91282CJZ59           645     6572948000  49,579.5793  US91282CJZ59   
 1       US912810TZ12           397     4395740000  52,589.4010  US912810TZ12   
 0       US912810TV08           586     6816956000  96,576.2865  US912810TV08   
 
            security   maturity     TTM  
 4  T 4 7/8 04/30/26 2026-04-30  1.9918  
 3  T 4 1/2 04/15/27 2027-04-15  2.9507  
 6  T 4 5/8 04/30/29 2029-04-30  4.9945  
 5  T 4 5/8 04/30/31 2031-04-30  6.9945  
 2      T 4 02/15/34 2034-02-15  9.7945  
 1  T 4 1/2 02/15/44 2044-02-15 19.8000  
 0  T 4 3/4 11/15/53 2053-11-15 29.5562  ,
 Figure({
   

## Problem 6 – Nelson–Siegel smooth hazard curve for ORCL
*6a* calibrate on-the-run Treasury curve. *6b* prepare ORCL fixed bonds (amt_out > 100). *6c* calibrate NS parameters.
*6d–e* compute model prices/yields, edges, and plot comparisons.

In [17]:
bond_sym = pd.read_excel(DATA / 'bond_symbology.xlsx')
bond_mkt = pd.read_excel(DATA / 'bond_market_prices_eod.xlsx')

run_sym = pd.read_excel(DATA / 'govt_on_the_run.xlsx')
run_details = run_sym.merge(bond_sym, on='isin', how='left')
run_market = bond_mkt.merge(run_sym[['isin']], on='isin')
run_market['midPrice'] = run_market[['bidPrice','askPrice']].mean(axis=1)
run_details = run_details.merge(run_market[['isin','midPrice']], on='isin')
run_details = run_details.sort_values('maturity')
run_details['class'] = 'Govt'

tsy_curve = cmt.calibrate_yield_curve_from_frame(ql_calc_date, run_details, 'midPrice')
tsy_curve_df = cmt.get_yield_curve_details_df(tsy_curve)

fig_tsy_zero = px.line(tsy_curve_df, x='YearFrac', y='ZeroRate', title='On-the-Run Treasury Zero Rates')
fig_tsy_df = px.line(tsy_curve_df, x='YearFrac', y='DiscountFactor', title='On-the-Run Treasury Discount Factors')
tsy_curve_df.head(), fig_tsy_zero, fig_tsy_df

(         Date  YearFrac  DiscountFactor  ZeroRate
 0  2024-05-03    0.0000          1.0000    4.9040
 1  2026-02-28    1.8330          0.9160    4.9040
 2  2026-03-31    1.9170          0.9130    4.8890
 3  2026-04-30    2.0000          0.9100    4.8540
 4  2027-02-15    2.7500          0.8790    4.7970,
 Figure({
     'data': [{'hovertemplate': 'YearFrac=%{x}<br>ZeroRate=%{y}<extra></extra>',
               'legendgroup': '',
               'line': {'color': '#636efa', 'dash': 'solid'},
               'marker': {'symbol': 'circle'},
               'mode': 'lines',
               'name': '',
               'orientation': 'v',
               'showlegend': False,
               'type': 'scatter',
               'x': {'bdata': ('AAAAAAAAAACHFtnO91P9P3npJjEIrP' ... '18P1U9QGiR7Xw/lT1AaJHtfD/VPUA='),
                     'dtype': 'f8'},
               'xaxis': 'x',
               'y': {'bdata': ('BFYOLbKdE0AEVg4tsp0TQHWTGARWjh' ... 'bz/dQSQN0kBoGVwxJALbKd76fGEkA='),
                     'd

In [18]:
orcl_sym = bond_sym[bond_sym['ticker'] == 'ORCL']
orcl_mkt = bond_mkt[bond_mkt['ticker'] == 'ORCL']
orcl_combined = orcl_sym.merge(orcl_mkt, on=['ticker','isin','figi','class'])
orcl_combined = orcl_combined[(orcl_combined['cpn_type']=='FIXED') & (orcl_combined['amt_out']>100)]
orcl_combined['midPrice'] = orcl_combined[['bidPrice','askPrice']].mean(axis=1)
orcl_combined['midYield'] = orcl_combined[['bidYield','askYield']].mean(axis=1)
orcl_combined = orcl_combined.dropna(subset=['midPrice'])
orcl_combined = orcl_combined.sort_values('maturity')
orcl_combined['TTM'] = (orcl_combined['maturity'] - calc_date).dt.days / 365

fig_orcl_yield = px.scatter(orcl_combined, x='TTM', y='midYield', title='ORCL Market Yields by TTM')
orcl_combined.head(), fig_orcl_yield

(   ticker class          figi          isin und_bench_tsy_isin  \
 23   ORCL  Corp  BBG00ZS0BNP1  US68389XCC74       US91282CKK61   
 11   ORCL  Corp  BBG00D7FYFT8  US68389XBM65       US91282CKK61   
 15   ORCL  Corp  BBG00SXGDG79  US68389XBU81       US91282CKJ98   
 12   ORCL  Corp  BBG00J5HRSQ6  US68389XBN49       US91282CKJ98   
 24   ORCL  Corp  BBG00ZS0BP03  US68389XCD57       US91282CKP58   
 
                security         name    type  coupon cpn_type  ... status  \
 23   ORCL 1.65 03/25/26  ORACLE CORP  GLOBAL  1.6500    FIXED  ...   ACTV   
 11   ORCL 2.65 07/15/26  ORACLE CORP  GLOBAL  2.6500    FIXED  ...   ACTV   
 15    ORCL 2.8 04/01/27  ORACLE CORP  GLOBAL  2.8000    FIXED  ...   ACTV   
 12  ORCL 3 1/4 11/15/27  ORACLE CORP  GLOBAL  3.2500    FIXED  ...   ACTV   
 24    ORCL 2.3 03/25/28  ORACLE CORP  GLOBAL  2.3000    FIXED  ...   ACTV   
 
          date  bidPrice askPrice accrued bidYield askYield midPrice midYield  \
 23 2024-05-03   93.3310  93.4620  0.1935   5

In [19]:
initial_params = (0.02, 0.01, -0.01, 1.0)
tsy_handle = ql.YieldTermStructureHandle(tsy_curve)
calib_results = cmt.calibrate_nelson_siegel_model(initial_params, ql_calc_date, orcl_combined, tsy_handle, bond_recovery_rate=0.4)
opt_params = calib_results.x

fixed_rate_bond_objects, calib_weights, bond_market_prices, bond_yields, bond_DV01s, bond_durations = cmt.create_bonds_and_weights(orcl_combined, tsy_handle)

bond_model_prices, bond_model_yields = cmt.calculate_nelson_siegel_model_prices_and_yields(opt_params, ql_calc_date, fixed_rate_bond_objects, tsy_handle, bond_recovery_rate=0.4)

orcl_results = orcl_combined.copy()
orcl_results['modelPrice'] = bond_model_prices
orcl_results['modelYield'] = bond_model_yields
orcl_results['edgePrice'] = orcl_results['midPrice'] - orcl_results['modelPrice']
orcl_results['edgeYield'] = orcl_results['midYield'] - orcl_results['modelYield']

params_df = pd.DataFrame({'parameter': ['theta1','theta2','theta3','lambda'], 'value': opt_params})
display(params_df)
print(latex_table(params_df))
orcl_results.head()

,parameter,value
0,theta1,0.0427
1,theta2,-0.0382
2,theta3,-0.0022
3,lambda,10.0000


```latex
\begin{document}
\begin{tabular}{|c|c|}
\hline
parameter & value \\ \hline
theta1 & 0.04270172474890133 \\ \hline
theta2 & -0.038166799432211775 \\ \hline
theta3 & -0.0022498396502255782 \\ \hline
lambda & 10.0 \\ \hline
\end{tabular}
\end{document}
```


,ticker,class,figi,isin,und_bench_tsy_isin,security,name,type,coupon,cpn_type,...,accrued,bidYield,askYield,midPrice,midYield,TTM,modelPrice,modelYield,edgePrice,edgeYield
23,ORCL,Corp,BBG00ZS0BNP1,US68389XCC74,US91282CKK61,ORCL 1.65 03/25/26,ORACLE CORP,GLOBAL,1.6500,FIXED,...,0.1935,5.4220,5.3440,93.3965,5.3830,1.8932,93.4296,5.3418,-0.0331,0.0412
11,ORCL,Corp,BBG00D7FYFT8,US68389XBM65,US91282CKK61,ORCL 2.65 07/15/26,ORACLE CORP,GLOBAL,2.6500,FIXED,...,0.8240,5.4030,5.3330,94.4530,5.3680,2.2000,94.7592,5.1999,-0.3062,0.1681
15,ORCL,Corp,BBG00SXGDG79,US68389XBU81,US91282CKJ98,ORCL 2.8 04/01/27,ORACLE CORP,GLOBAL,2.8000,FIXED,...,0.2800,5.2490,5.1820,93.5730,5.2155,2.9123,93.4055,5.2722,0.1675,-0.0567
12,ORCL,Corp,BBG00J5HRSQ6,US68389XBN49,US91282CKJ98,ORCL 3 1/4 11/15/27,ORACLE CORP,GLOBAL,3.2500,FIXED,...,1.5530,5.2360,5.1640,93.7960,5.2000,3.5370,93.8153,5.1878,-0.0193,0.0122
24,ORCL,Corp,BBG00ZS0BP03,US68389XCD57,US91282CKP58,ORCL 2.3 03/25/28,ORACLE CORP,GLOBAL,2.3000,FIXED,...,0.2680,5.2750,5.2150,89.7720,5.2450,3.8959,89.7961,5.2292,-0.0241,0.0158


In [20]:
fig_price_comp = px.scatter(orcl_results, x='maturity', y=['midPrice','modelPrice'], title='ORCL Model vs Market Prices')
fig_yield_comp = px.scatter(orcl_results, x='maturity', y=['midYield','modelYield'], title='ORCL Model vs Market Yields')
fig_edge_yield = px.bar(orcl_results, x='maturity', y='edgeYield', title='ORCL Yield Edges')

fig_price_comp, fig_yield_comp, fig_edge_yield

(Figure({
     'data': [{'hovertemplate': 'variable=midPrice<br>maturity=%{x}<br>value=%{y}<extra></extra>',
               'legendgroup': 'midPrice',
               'marker': {'color': '#636efa', 'symbol': 'circle'},
               'mode': 'markers',
               'name': 'midPrice',
               'orientation': 'v',
               'showlegend': True,
               'type': 'scatter',
               'x': array(['2026-03-25T00:00:00.000000000', '2026-07-15T00:00:00.000000000',
                           '2027-04-01T00:00:00.000000000', '2027-11-15T00:00:00.000000000',
                           '2028-03-25T00:00:00.000000000', '2028-05-06T00:00:00.000000000',
                           '2029-11-09T00:00:00.000000000', '2030-04-01T00:00:00.000000000',
                           '2030-05-06T00:00:00.000000000', '2030-05-15T00:00:00.000000000',
                           '2031-03-25T00:00:00.000000000', '2032-11-09T00:00:00.000000000',
                           '2033-02-06T00:00:00.000